In [1]:
import os
os.chdir('../..')

In [2]:
import sys
import pandas as pd

import pickle

import matplotlib.pyplot as plot

from os import listdir
from os.path import isfile, join

import lightgbm as lgb

import glob

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import OrdinalEncoder
from modules.ml_efficacy.LGBMOrdinal_base import LGBMOrdinal, LGBMRegressor, LGBMClassifier, LightGBMCV, emse, emae

import warnings
warnings.filterwarnings('ignore')

# For the Python notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Functions

In [3]:
def process_real_df(df, cat_cols=None, ord_cols=None, cont_cols=None, enc=None):
    if cat_cols and cont_cols:
        df[cat_cols+ord_cols] = enc.fit_transform(df[cat_cols+ord_cols])
    else:
        print('Automated inference of column types to be implemented!')
    return df

def process_syn_df(df, cat_cols, ord_cols, cont_cols, enc=None):
    df[cat_cols+ord_cols] = enc.transform(df[cat_cols+ord_cols])

    return df

def check_low_appearing_vars(df):
    
    for c in df.columns:
        val = df[c].value_counts()
        if len(val) < 20:
            val = val/len(df)
            if any(val < 0.01) and c != 'choice':
                print(c)
                print(val)
                print()
                
def replace_low_appearing_values(df, dataset):
    
    if 'Chicago' in dataset:
        dct_ = {}
        for i in df['hh_vehicles'].unique():
            if i >= 5:
                dct_[i] = '5+'
            else:
                dct_[i] = str(i)        
        df['hh_vehicles'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_size'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_size'].replace(dct_, inplace=True)
        
        dct_ = {}
        for i in df['hh_bikes'].unique():
            if i >= 6:
                dct_[i] = '6+'
            else:
                dct_[i] = str(i)        
        df['hh_bikes'].replace(dct_, inplace=True)   
    
    elif 'LPMC' in dataset:
        dct_ = {}
        for i in df['pt_n_interchanges'].unique():
            if i >= 2:
                dct_[i] = '2+'
            else:
                dct_[i] = str(i)        
        df['pt_n_interchanges'].replace(dct_, inplace=True) 
        
        dct_ = {
            'Diesel_LGV': 'LGV',
            'Petrol_LGV': 'LGV',
            'Hybrid_Car': 'Average_Car'
        }
        df['fueltype'].replace(dct_, inplace=True)

# Compute results

In [4]:
dataset = 'LPMC'

input_folder = '../synth_data/TEST/'

files_ = {}
models = []

for f in listdir(input_folder):
    if isfile(join(input_folder, f)):
        m = f.split('.')[0]
        models.append(m)
        files_[m] = join(input_folder, f)
        
models.append('original')
files_['original'] = '../data/' + dataset.split('_')[0] + '/data.csv' 

In [5]:
models

['CTABGAN', 'CTGAN', 'DATGAN', 'original']

In [6]:
df_orig = pd.read_csv('../data/' + dataset.split('_')[0] + '/data.csv')

In [7]:
df_orig.head()

,travel_mode,purpose,fueltype,faretype,bus_scale,travel_year,travel_month,travel_date,day_of_week,start_time_linear,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_n_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_con_charge,driving_traffic_percent
0,drive,HBO,Petrol_Car,child,0.0,2012,4,1,7,10.000000,...,0.134444,0.0,0.016667,0.000000,0,0.052222,0.0,0.14,0.0,0.111702
1,drive,HBO,Petrol_Car,free,0.0,2012,4,1,7,17.666667,...,0.241389,0.0,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126
2,drive,HBO,Petrol_Car,full,1.0,2012,4,1,7,13.000000,...,0.257500,0.0,0.873889,0.089444,1,0.508333,3.0,1.59,0.0,0.356831
3,pt,HBW,Average_Car,full,1.0,2012,4,1,7,21.500000,...,0.123889,0.0,0.208056,0.091667,1,0.115556,3.0,0.33,0.0,0.033654
4,pt,HBO,Average_Car,free,0.0,2012,4,1,7,8.916667,...,0.171389,0.0,0.334444,0.000000,0,0.196389,0.0,0.53,0.0,0.035361


In [8]:
replace_low_appearing_values(df_orig, dataset)

In [9]:
check_low_appearing_vars(df_orig)

In [10]:
if 'Chicago' in dataset:
    cont_cols = ['distance', 'age', 'departure_time']
    ord_cols = ['hh_vehicles', 'hh_size', 'hh_bikes', 'hh_income', 'education_level']
    cat_cols = [col for col in df_orig.columns if col not in cont_cols + ord_cols]
elif 'LPMC' in dataset:
    cont_cols = ['start_time_linear', 'age', 'distance', 'dur_walking', 
                 'dur_cycling', 'dur_pt_access', 'dur_pt_rail', 'dur_pt_bus', 
                 'dur_pt_int', 'dur_driving', 'cost_transit', 
                 'cost_driving_fuel', 'driving_traffic_percent']
    ord_cols = ['travel_year', 'travel_month', 'travel_date', 
                'day_of_week', 'pt_n_interchanges', 'car_ownership']
    cat_cols = [col for col in df_orig.columns if col not in cont_cols + ord_cols]

In [11]:
enc = OrdinalEncoder()
ori = process_real_df(df_orig, cat_cols, ord_cols, cont_cols, enc)

In [12]:
filepath = './notebooks/tests/ml_efficacy/'
filename = 'cv_result.pickle'
cv_modelscores = {}
params={'n_estimators': 5000}

try:
    cv_modelscores = pickle.load(open(f'{filepath}{filename}','rb'))
    print('Found previous pickel file, using that')
except:
    print('No previous results found, starting fresh')
    try:
        os.makedirs(filepath)
    except:
        pass

No previous results found, starting fresh


In [13]:
for i, model in enumerate(models):
    
    tmp_df = pd.read_csv(files_[model])
    replace_low_appearing_values(tmp_df, dataset)
    v_df = process_syn_df(tmp_df, cat_cols, ord_cols, cont_cols, enc)
        
    if model in cv_modelscores:
        print(f'Previous results for {model}')
    else:
        print(f'Getting results for {model}')
        cv_modelscores[model] = {}
        for j, ycol in enumerate(ori.columns):
            info = '    Column: {} ({}/{})'.format(ycol, j+1, len(ori.columns))
            print(info, end="")
            sys.stdout.flush()
            
            Xcols = [c for c in ori.columns if c!=ycol]

            y_synth = v_df[ycol]
            X_synth = v_df[Xcols]
            y_real = ori[ycol]
            X_real = ori[Xcols]


            observe_sets = {'original': (X_real, y_real)}
            ccols = [c for c in cat_cols if c!=ycol]


            if ycol in cat_cols + ord_cols:
                lgbm_type = 'LGBMClassifier'
                kf = StratifiedKFold(shuffle=True, random_state=42)
                eval_metric = ['error']
            elif ycol in cont_cols:
                lgbm_type = 'LGBMRegressor'
                kf = KFold(shuffle=True, random_state=42)
                eval_metric = ['l2', 'l1']
            cv = LightGBMCV(lgbm_type=lgbm_type,
                splitter = kf,
                eval_metric = eval_metric,
                observe_sets = observe_sets,
                separate_observation_split = True,
                early_stopping_rounds = 5,
                return_cv_models = False,
                refit_model = False,
                verbose = True)
            cv.fit(X_synth, y_synth, categorical_feature=ccols, params=params)
            cv_modelscores[model][ycol] = cv.result_dict_
            
            print(' '*len(info), end='\r')

            if j == len(ori.columns):
                print('', end='\r')
            
        pickle.dump(cv_modelscores,open(f'{filepath}/{filename}','wb'))

        
print("\033[1mFINISHED!\033[0m")

Getting results for CTABGAN
Getting results for CTGAN                  
Getting results for DATGAN                 
Getting results for original               
FINISHED!                          


In [14]:
#cv_modelscores.pop('CTABGAN')
#pickle.dump(cv_modelscores,open(f'{filepath}/{filename}','wb'))

In [15]:
internal = {}
external = {}
external_normalised = {}
cont_scores = {}
cat_scores = {}
ori_scores = {col: cv_modelscores['original'][col]['test_log_loss'] for col in cat_cols + ord_cols}
ori_scores.update({col: cv_modelscores['original'][col]['test_l2'] for col in cont_cols})
for model in models:
    internal[model] = {col: cv_modelscores[model][col]['test_log_loss'] for col in cat_cols + ord_cols}
    external[model] = {col: cv_modelscores[model][col]['original_log_loss'] for col in cat_cols + ord_cols}
    external_normalised[model] = {col: external[model][col]-ori_scores[col] for col in cat_cols + ord_cols}
    
    internal[model].update({col: cv_modelscores[model][col]['test_l2'] for col in cont_cols})
    external[model].update({col: cv_modelscores[model][col]['original_l2'] for col in cont_cols})
    external_normalised[model].update({col: external[model][col]/ori_scores[col] for col in cont_cols})
    
    cont_scores[model] = sum([external[model][col]/ori_scores[col] for col in cont_cols])
    cat_scores[model] = sum([external[model][col]-ori_scores[col] for col in cat_cols + ord_cols])

In [16]:
cat_sorted = sorted(cat_scores.items(), key=lambda item: item[1])
cont_sorted = sorted(cont_scores.items(), key=lambda item: item[1])

In [17]:
i=1
print('   | {:<30} | {:<30}'.format('categorical', 'continuous'))
print('-----------------------------------------------------------')
for a, b in zip(cat_sorted, cont_sorted):
    print('{:>2} | {:<30} | {:<30}'.format(i, '{:<12}: {:.3f}'.format(a[0], a[1]), '{:<12}: {:.3f}'.format(b[0], b[1])))
    i+=1

   | categorical                    | continuous                    
-----------------------------------------------------------
 1 | original    : -3.596           | original    : 9.249           
 2 | DATGAN      : 2.243            | DATGAN      : 21.907          
 3 | CTGAN       : 2.904            | CTABGAN     : 33.756          
 4 | CTABGAN     : 5.595            | CTGAN       : 39.943          
